任务9内容：

* 完成第11章的学习，代码实现的过程上传到GitHub

DDL：20190827 08:00am

# 第11章 时间序列

时间序列数据类型：
* 时间戳（timestamp)
* 固定时期（period）
* 时间间隔（interval）

In [7]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
# np.random.seed(2018)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# plt.rc('figure', figsize=(10, 6))
# PREVIOUS_MAX_ROWS = pd.options.display.max_rows
# pd.options.display.max_rows = 20
# np.set_printoptions(precision=4, suppress=True)

## 11.1 日期和日期数据类型及工具

datetime模块中的数据类型：
* date
* time
* datetime
* timedelta

In [2]:
from datetime import datetime

In [3]:
# 获取现在的时间
now = datetime.now()
now

datetime.datetime(2019, 8, 26, 21, 25, 43, 240029)

In [6]:
now.year, now.month, now.day

(2019, 8, 26)

In [7]:
# datetime以毫秒形式存储日期和时间
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(926, 56700)

In [9]:
delta.days, delta.seconds

(926, 56700)

In [12]:
# timedelta(): 表示两个datetime值之间的差（日、秒、毫秒）
from datetime import timedelta
start = datetime(2011, 1, 7)
start + timedelta(12), start - 2 * timedelta(12)

(datetime.datetime(2011, 1, 19, 0, 0), datetime.datetime(2010, 12, 14, 0, 0))

In [11]:
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

### 11.1.1 字符串和datetime的相互转换

* datetime对象和pd.timestamp对象转成字符串：
    * str()
    * strftime


* 字符串转成日期：
    * datetime.strptime()
    * dateutil.parser.parse()
    * pd.to_datetime()

datetime对象和pd.timestamp对象转成字符串

In [13]:
stamp = datetime(2011, 1, 3)
str(stamp)

'2011-01-03 00:00:00'

In [18]:
stamp.strftime('%Y-%m-%d') # datetime格式定义P339

'2011-01-03-00-00'

字符串转成日期

In [19]:
# datetime.strptime()
# 要编写日期格式
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [20]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

In [1]:
# dateutil.parser.parse()
# 不必编写日期格式
# 可以解析几乎所有人类能够理解的日期表示形式
from dateutil.parser import parse
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [2]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

In [5]:
parse('6/12/2011')

datetime.datetime(2011, 6, 12, 0, 0)

In [4]:
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

In [8]:
# pd.to_datetime()
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

In [12]:
type(pd.to_datetime(datestrs))

pandas.core.indexes.datetimes.DatetimeIndex

In [9]:
# pd.to_datetime()处理缺失值（None, 空字符串等）
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [10]:
idx[2] # Not a Time -- pandas中时间戳数据的null值

NaT

In [11]:
pd.isnull(idx)

array([False, False,  True])

## 11.2 时间序列基础

pandas最基本的时间序列类型就是以时间戳为索引的Series


In [15]:
from datetime import datetime
np.random.seed(2018)
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02   -0.276768
2011-01-05    0.581851
2011-01-07    2.148399
2011-01-08   -1.279487
2011-01-10    0.502277
2011-01-12    0.856029
dtype: float64

In [16]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [19]:
ts.index.dtype

dtype('<M8[ns]')

In [17]:
# 时间序列之间的算术运算会自动按日期对齐
ts + ts[::2]

2011-01-02   -0.553535
2011-01-05         NaN
2011-01-07    4.296799
2011-01-08         NaN
2011-01-10    1.004554
2011-01-12         NaN
dtype: float64

In [20]:
# DatetimeIndex中的各个标量值是pandas的Timestamp对象
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

### 11.2.1 索引、选取、子集构造

* df[ ], series[ ] 这种切片产生的是源时间序列的视图，意味着没有数据被复制，对切片进行修改会反应到原始数据上
* df.truncate(), series.truncate()

In [32]:
ts

2011-01-02   -0.276768
2011-01-05    0.581851
2011-01-07    2.148399
2011-01-08   -1.279487
2011-01-10    0.502277
2011-01-12    0.856029
dtype: float64

In [30]:
stamp = ts.index[2]
stamp

Timestamp('2011-01-07 00:00:00')

In [31]:
ts[stamp]

2.14839926388642

In [33]:
ts['2011-01-07']

2.14839926388642

In [34]:
ts[2]

2.14839926388642

In [35]:
longer_ts = pd.Series(np.random.randn(1000),
                      index=pd.date_range('1/1/2000', periods=1000))
longer_ts

2000-01-01   -0.142790
2000-01-02    0.110079
2000-01-03   -0.688065
2000-01-04    0.433564
2000-01-05    0.510221
2000-01-06   -0.165131
2000-01-07   -1.351779
2000-01-08    0.546631
2000-01-09    1.230655
2000-01-10    1.076446
2000-01-11   -1.210625
2000-01-12   -0.306677
2000-01-13   -1.057419
2000-01-14    0.402057
2000-01-15    0.289165
2000-01-16    1.282733
2000-01-17   -1.065696
2000-01-18   -1.706633
2000-01-19   -0.172797
2000-01-20    0.063710
2000-01-21    0.370628
2000-01-22   -1.604543
2000-01-23   -2.165729
2000-01-24    0.380370
2000-01-25   -0.276501
2000-01-26   -0.575682
2000-01-27    0.421854
2000-01-28    0.704780
2000-01-29    0.045401
2000-01-30    0.530690
                ...   
2002-08-28    0.779939
2002-08-29    0.542527
2002-08-30    0.112462
2002-08-31    0.476439
2002-09-01    1.380218
2002-09-02   -1.143243
2002-09-03    1.225896
2002-09-04   -0.637136
2002-09-05    0.754269
2002-09-06    0.909113
2002-09-07    3.326698
2002-09-08    1.834776
2002-09-09 

In [37]:
# 获取某年数据切片
longer_ts['2001']

2001-01-01    0.697280
2001-01-02    0.990864
2001-01-03    0.521882
2001-01-04    0.625746
2001-01-05    0.778274
2001-01-06   -0.249089
2001-01-07   -2.226924
2001-01-08    1.180519
2001-01-09    0.238895
2001-01-10    0.317275
2001-01-11   -0.015794
2001-01-12   -1.205670
2001-01-13    0.378758
2001-01-14   -2.611850
2001-01-15   -0.356728
2001-01-16    2.011076
2001-01-17    0.894436
2001-01-18   -1.050479
2001-01-19   -0.005631
2001-01-20   -0.607471
2001-01-21    0.783794
2001-01-22   -0.823705
2001-01-23    0.120829
2001-01-24   -0.143681
2001-01-25    0.392827
2001-01-26    0.746446
2001-01-27    0.558511
2001-01-28    0.289991
2001-01-29    1.234559
2001-01-30    0.464686
                ...   
2001-12-02    1.255126
2001-12-03    1.270203
2001-12-04   -1.610575
2001-12-05    0.429620
2001-12-06    0.939115
2001-12-07    0.034363
2001-12-08   -1.005493
2001-12-09   -0.144789
2001-12-10   -0.324071
2001-12-11   -0.137565
2001-12-12   -0.883033
2001-12-13    0.392313
2001-12-14 

In [39]:
# 获取某年某月数据切片
longer_ts['2001-05']

2001-05-01   -1.146121
2001-05-02   -0.032866
2001-05-03    1.535003
2001-05-04    0.068195
2001-05-05   -0.104443
2001-05-06   -0.126879
2001-05-07   -0.250956
2001-05-08    1.043293
2001-05-09    0.705272
2001-05-10    0.928829
2001-05-11   -0.839979
2001-05-12    0.268778
2001-05-13   -0.423253
2001-05-14    0.865299
2001-05-15    0.864023
2001-05-16   -1.723836
2001-05-17   -0.718277
2001-05-18   -0.674208
2001-05-19    0.205657
2001-05-20    0.426447
2001-05-21    0.954524
2001-05-22    0.375241
2001-05-23    1.642820
2001-05-24    0.329666
2001-05-25    0.373081
2001-05-26   -1.231775
2001-05-27   -0.547934
2001-05-28    1.378574
2001-05-29   -0.836000
2001-05-30   -1.135351
2001-05-31    0.160416
Freq: D, dtype: float64

In [41]:
ts

2011-01-02   -0.276768
2011-01-05    0.581851
2011-01-07    2.148399
2011-01-08   -1.279487
2011-01-10    0.502277
2011-01-12    0.856029
dtype: float64

In [40]:
# 使用datetime对象进行切片
ts[datetime(2011, 1, 7):]

2011-01-07    2.148399
2011-01-08   -1.279487
2011-01-10    0.502277
2011-01-12    0.856029
dtype: float64

In [43]:
ts['2011-01-05':'2011-01-10']

2011-01-05    0.581851
2011-01-07    2.148399
2011-01-08   -1.279487
2011-01-10    0.502277
dtype: float64

In [44]:
# truncate()方法
ts.truncate(after='1/9/2011')

2011-01-02   -0.276768
2011-01-05    0.581851
2011-01-07    2.148399
2011-01-08   -1.279487
dtype: float64

In [45]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')
long_df = pd.DataFrame(np.random.randn(100, 4),
                       index=dates,
                       columns=['Colorado', 'Texas',
                                'New York', 'Ohio'])
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,1.308551,0.762055,1.546174,-0.614282
2001-05-09,0.106805,0.178052,-0.030801,0.123041
2001-05-16,0.439814,0.870248,0.128192,0.342858
2001-05-23,0.517524,-0.767912,0.295127,0.299915
2001-05-30,0.356533,0.851797,-0.366819,0.378114


### 11.2.2 带有重复索引的时间序列

* 检查索引是否重复：df.index.is_unique
* 对具有非唯一时间戳的数据进行聚合：df.groupby(level=0)


In [46]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

In [47]:
# df.index.is_unique
dup_ts.index.is_unique

False

In [55]:
# df.groupby(level=0)
grouped = dup_ts.groupby(level=0)
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

In [56]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3 时间的范围、频率以及移动

In [57]:
ts

2011-01-02   -0.276768
2011-01-05    0.581851
2011-01-07    2.148399
2011-01-08   -1.279487
2011-01-10    0.502277
2011-01-12    0.856029
dtype: float64

In [63]:
# 将不规则的时间序列转换为具有固定频率的时间序列：df.resample(), series.resample()
resampler = ts.resample('D') # 'D'是每天的意思
resampler

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

### 11.3.1 生成日期范围

* pandas.date_range()

In [64]:
index = pd.date_range('2012-04-01', '2012-06-01')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [68]:
pd.date_range(start='2012-04-01', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [69]:
pd.date_range(end='2012-06-01', periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [70]:
pd.date_range('2000-01-01', '2000-12-01', freq='BM') # BM = business end of month

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

部分时间序列频率P360

In [71]:
pd.date_range('2012-05-02 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [72]:
# normalize=True 时间戳被规范到午夜
pd.date_range('2012-05-02 12:56:31', periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 11.3.2 频率和日期偏移量

pandas中的频率组成：
* 基础频率：'M'(每月)，'H'(每小时)
* 乘数

In [73]:
from pandas.tseries.offsets import Hour, Minute
# 按小时计算的频率：Hour类
hour = Hour()
hour

<Hour>

In [74]:
# 定义偏移量的倍数
four_hours = Hour(4)
four_hours

<4 * Hours>

In [75]:
# 生成日期范围，指定日期频率和偏移量
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [76]:
# 偏移量对象可通过加法进行连接
Hour(2) + Minute(30)

<150 * Minutes>

In [77]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

pandas中的频率代码和日期偏移类P354

### 11.3.3 WOM日期

WOM（Week of Month），频率类，以WOM开头

In [81]:
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI') # 每月第三个星期五
rng

DatetimeIndex(['2012-01-20', '2012-02-17', '2012-03-16', '2012-04-20',
               '2012-05-18', '2012-06-15', '2012-07-20', '2012-08-17'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

In [82]:
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

### 11.3.4 移动（超前和滞后）数据

* 单纯的前移和后移操作：series.shift(), df.shift()

In [92]:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2000', periods=4, normalize=True, freq='M'))
ts

2000-01-31    0.522094
2000-02-29   -0.993709
2000-03-31    1.937354
2000-04-30    1.198675
Freq: M, dtype: float64

In [93]:
ts.shift(2) # 时间戳不动，值后移

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.522094
2000-04-30   -0.993709
Freq: M, dtype: float64

In [94]:
ts.shift(-2) # 时间戳不动，值前移

2000-01-31    1.937354
2000-02-29    1.198675
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

In [95]:
# 已知频率，可实现时间戳位移，而不是对数据进行简单位移
ts.shift(2, freq='M')

2000-03-31    0.522094
2000-04-30   -0.993709
2000-05-31    1.937354
2000-06-30    1.198675
Freq: M, dtype: float64

In [96]:
ts.shift(3, freq='D')

2000-02-03    0.522094
2000-03-03   -0.993709
2000-04-03    1.937354
2000-05-03    1.198675
dtype: float64

In [98]:
ts.shift(1, freq='90T') # 90T: 每90分钟

2000-01-31 01:30:00    0.522094
2000-02-29 01:30:00   -0.993709
2000-03-31 01:30:00    1.937354
2000-04-30 01:30:00    1.198675
Freq: M, dtype: float64

### 11.3.5 通过偏移量对日期进行位移

* 使用算术运算 +
* 锚点偏移量的rollforward（日期向前滚动）和rollback（日期向后滚动）

In [99]:
from pandas.tseries.offsets import Day, MonthEnd
now = datetime(2011, 11, 17)
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

In [100]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [102]:
# rollforward()
offset = MonthEnd()
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [103]:
# rollback()
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

In [104]:
ts = pd.Series(np.random.randn(20),
               index=pd.date_range('1/15/2000', periods=20, freq='4d'))
ts

2000-01-15   -0.354862
2000-01-19   -0.425840
2000-01-23   -0.682284
2000-01-27   -0.165488
2000-01-31    1.153016
2000-02-04   -0.717749
2000-02-08   -1.353969
2000-02-12   -0.819575
2000-02-16    0.678480
2000-02-20    0.511985
2000-02-24   -0.330796
2000-02-28   -1.572501
2000-03-03    1.212696
2000-03-07   -1.372621
2000-03-11   -0.096371
2000-03-15   -1.936695
2000-03-19   -1.594045
2000-03-23   -0.453298
2000-03-27   -1.281298
2000-03-31    1.785441
Freq: 4D, dtype: float64

In [115]:
# rollforward(), rollback() 结合groupby
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.095091
2000-02-29   -0.514875
2000-03-31   -0.467024
dtype: float64

In [111]:
ofs_rf = []
for i in ts.index:
    val = offset.rollforward(i)
    ofs_rf.append(val)

ofs_rf

[Timestamp('2000-01-31 00:00:00'),
 Timestamp('2000-01-31 00:00:00'),
 Timestamp('2000-01-31 00:00:00'),
 Timestamp('2000-01-31 00:00:00'),
 Timestamp('2000-01-31 00:00:00', freq='4D'),
 Timestamp('2000-02-29 00:00:00'),
 Timestamp('2000-02-29 00:00:00'),
 Timestamp('2000-02-29 00:00:00'),
 Timestamp('2000-02-29 00:00:00'),
 Timestamp('2000-02-29 00:00:00'),
 Timestamp('2000-02-29 00:00:00'),
 Timestamp('2000-02-29 00:00:00'),
 Timestamp('2000-03-31 00:00:00'),
 Timestamp('2000-03-31 00:00:00'),
 Timestamp('2000-03-31 00:00:00'),
 Timestamp('2000-03-31 00:00:00'),
 Timestamp('2000-03-31 00:00:00'),
 Timestamp('2000-03-31 00:00:00'),
 Timestamp('2000-03-31 00:00:00'),
 Timestamp('2000-03-31 00:00:00', freq='4D')]

In [112]:
# 以上groupby + rollforward 的方法等价于
# resample
ts.resample('M').mean()

2000-01-31   -0.095091
2000-02-29   -0.514875
2000-03-31   -0.467024
Freq: M, dtype: float64

## 11.4 时区处理

In [120]:
# 导入时区处理相关库
import pytz
# 查询时区划分最后5个国家
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [121]:
# 获取时区对象：pytz.timezone
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

### 11.4.1 时区本地化和转换

In [123]:
np.random.seed(2018)
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-09 09:30:00   -0.276768
2012-03-10 09:30:00    0.581851
2012-03-11 09:30:00    2.148399
2012-03-12 09:30:00   -1.279487
2012-03-13 09:30:00    0.502277
2012-03-14 09:30:00    0.856029
Freq: D, dtype: float64

In [124]:
print(ts.index.tz) # 此时索引的ts字段为None

None


In [127]:
# 用时区集生成日期范围
np.random.seed(2018)
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [128]:
# 单纯到本地化的转换：tz_localize
ts

2012-03-09 09:30:00   -0.276768
2012-03-10 09:30:00    0.581851
2012-03-11 09:30:00    2.148399
2012-03-12 09:30:00   -1.279487
2012-03-13 09:30:00    0.502277
2012-03-14 09:30:00    0.856029
Freq: D, dtype: float64

In [130]:
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00   -0.276768
2012-03-10 09:30:00+00:00    0.581851
2012-03-11 09:30:00+00:00    2.148399
2012-03-12 09:30:00+00:00   -1.279487
2012-03-13 09:30:00+00:00    0.502277
2012-03-14 09:30:00+00:00    0.856029
Freq: D, dtype: float64

In [131]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [132]:
# 转换成其他时区的时间：tz.convert()
ts_utc.tz_convert('America/New_York')

2012-03-09 04:30:00-05:00   -0.276768
2012-03-10 04:30:00-05:00    0.581851
2012-03-11 05:30:00-04:00    2.148399
2012-03-12 05:30:00-04:00   -1.279487
2012-03-13 05:30:00-04:00    0.502277
2012-03-14 05:30:00-04:00    0.856029
Freq: D, dtype: float64

In [133]:
ts_eastern = ts.tz_localize('America/New_York')
ts_eastern

2012-03-09 09:30:00-05:00   -0.276768
2012-03-10 09:30:00-05:00    0.581851
2012-03-11 09:30:00-04:00    2.148399
2012-03-12 09:30:00-04:00   -1.279487
2012-03-13 09:30:00-04:00    0.502277
2012-03-14 09:30:00-04:00    0.856029
Freq: D, dtype: float64

In [134]:
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00   -0.276768
2012-03-10 14:30:00+00:00    0.581851
2012-03-11 13:30:00+00:00    2.148399
2012-03-12 13:30:00+00:00   -1.279487
2012-03-13 13:30:00+00:00    0.502277
2012-03-14 13:30:00+00:00    0.856029
Freq: D, dtype: float64

In [136]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00   -0.276768
2012-03-10 15:30:00+01:00    0.581851
2012-03-11 14:30:00+01:00    2.148399
2012-03-12 14:30:00+01:00   -1.279487
2012-03-13 14:30:00+01:00    0.502277
2012-03-14 14:30:00+01:00    0.856029
Freq: D, dtype: float64

### 11.4.2 操作时区意识型Timestamp对象

### 11.4.3 不同时区之间的运算

## 11.5 十七及其算术运算

### 11.5.1 时期的频率转换

### 11.5.2 按季度计算的时期频率

### 11.5.3 将Timestamp转换为Period（及其反向过程）

### 11.5.4 通过数组创建PeriodIndex

## 11.6 重采样及频率转换

### 11.6.1 降采样

### 11.6.2 OHLC重采样

### 11.6.3 升采样和差值

### 11.6.4 通过时期进行重采样

## 11.7 移动窗口函数

### 11.7.1 指数加权函数

### 11.7.2 二元移动窗口函数

### 11.7.3 用户定义的移动窗口函数